# Projekt założenia
symulator zdarzeniowy M/M/1/N
    - średni czas oczekiwania na obsługę
    - prawdopobieństwo blokady
porównać zmodelem analitycznym:
    - dla N = 51
    - różnych wartości obciążenia: 0.01, 0.05, 0.1

średni czas oczekiwania - > histogram


In [155]:
# https://pyshark.com/poisson-distribution-and-poisson-process-in-python/#poisson-distribution-example-in-python
import scipy
import numpy as np
import matplotlib
from enum import Enum
import uuid

czas w serwerze to jest "usługa" 

In [160]:
# global_time = 0
n = 51 # max order in a system
# zmienna lososowa opisująca czas między zgłoszeniami
la = 3.35 # 1.0478 # 0.977 #  # intensywność napływu
# time_between_packet = 1 / la
# zmienna losowa opisuje czas pomiędzy zgłoszeniami k przez serwer
mu = 3
# time_of_service = 1 / mu # intensywność obsługi
time_of_simulation = 10
np.random.seed(12)

# https://github.com/amasl2048/queue-mm1/blob/master/qtest.cpp
roh = la / mu
rn = pow(roh, n + 1)
p0 = (1 - roh) / (1 - rn)         # free queue probability
pb = p0 * rn / roh                

navg = roh / (1-roh) - (n+1) * rn/(1-rn)  
tavg = navg / la                    
u    = mu * (1 - p0)                

out = '''
pb   = %.4f    # block probability
navg = %.4f    # average orders in a system
tavg = %.4f    # average time in a system
u    = %.4f    # throughput
'''
print(out % (pb, navg, tavg, u))


pb   = 0.1048    # block probability
navg = 42.5966    # average orders in a system
tavg = 12.7154    # average time in a system
u    = 2.9989    # throughput



In [157]:
class EventType(Enum):
    IN_EVENT = "IN_EVENT"
    OUT_EVENT = "OUT_EVENT"

# slajd 91, 

class Event():
    def __init__(self, time: float, event_type, client_id = None):
        self.time = time
        self.event_type = event_type
        if client_id == None:
            self.client_id = str(uuid.uuid4())
        else:
            self.client_id = client_id
    
    def __str__(self):
        mes = f"{self.client_id}, {self.event_type}, {self.time}"
        return mes
    
    def set_time_in_queue(self, time_in_queue):
        self.time_in_queue = time_in_queue
    
    def get_event_stats(self):
        return {
            'oc_time': self.time,
            'qu_time': self.time_in_queue,
            'client_id': self.client_id,
            'event_type': self.event_type
        }

In [181]:
class Server:
    def __init__(self):
        self.all_clients =0 
        self.all_clients_in_queue = 0
        self.client_in_server = False
        self.clients_number_in_queue = 0 
        self.client_in_server = None
        self.clients_in_queue = []
        self.buffor = n
    
    def get_server_stats(self):
        return self.client_in_server, self.clients_number_in_queue

    def is_server_empty(self):
        return not self.clients_in_server

    def is_queue_empty(self):
        return not self.clients_in_queue

    def is_place_for_new_event(self):
        return self.clients_number_in_queue < self.buffor

    def add_client_to_server(self, client_id, timestamp):
        self.all_clients += 1
        self.client_in_server = True
        self.clients_in_server = (client_id, timestamp)

    def add_client_to_queue(self, client_id, timestamp):
        self.all_clients_in_queue += 1
        self.clients_number_in_queue += 1
        self.clients_in_queue.insert(0, (client_id, timestamp))

    def get_queue_first_client(self):
        return self.clients_in_queue[-1]


    def delete_client_from_queue(self, client_id):
        self.clients_number_in_queue -= 1
        self.clients_in_queue = list(
            filter(
                lambda client: client[0] != client_id,
                self.clients_in_queue))

    def delete_client_from_server(self):
        self.client_in_server = False
        self.client_in_server = None


    """


s = Server()
for i in range(53):
    s.add_client(str(i), str(i))
print(s.buffor)


51


In [185]:
# https://github.com/gsharma007/Stochastic_OR_and_Simulation_Modeling/blob/master/Queue_Modeling/DES.py
import logging

class Simulation:
    def __init__(self):
        # marge
        self.events = []
        self.clients_stats = []
        self.server = Server()
        self.clients = 0 
        # init logging
        

    def handle_event(self):
        current_event = self.events.pop()
        current_time = current_event.time
        # loggin
        if current_event.event_type == EventType.IN_EVENT:
            pass
            # make out event 
            # check buffor
            #

    def init_event(self):
        pass

    def run_simulation(self):
        self.init_event()
    
    def generate_interarrival(self):
        return np.random.exponential(la)
    
    def generate_service(self):
        return np.random.exponential(mu)


get pobiera zdarzenie z listy z namnijeszym czasem, przesuwa czas symulacji do tego czasu

To do: 
- ogarnać czy potrzebny jest server
    - liczba n ?
- dodac logging do servera lub klientów
- zmienić symualcje od maksymalnego czasu, a nie iteracji 
- zrobić jakieś wykresy od lambda ? 
- zrobić historgram dla b
- sprawko 
- wysłać

UAKTUALNIENIE: ponieważ w punkcie III projektu wkradł się błąd a jest już blisko terminu oddania prac, proszę wykonać tę część przy prostym założeniu, że odstęp miedzy przychodzącymi klientami jest stały i równy średniej z punktu I. 
Reference materials